In [0]:
# Creacion de los schemas y tablas requeridos

In [0]:
schemas_list = ['bronze','silver','gold']



In [0]:

# Limpieza de tablas
spark.sql("USE CATALOG dev_ecommerce")

spark.sql(" DROP SCHEMA IF EXISTS dev_ecommerce.golden CASCADE")
spark.sql(" DROP SCHEMA IF EXISTS dev_ecommerce.silver CASCADE")
spark.sql(" DROP SCHEMA IF EXISTS dev_ecommerce.bronze CASCADE")
spark.sql(" DROP CATALOG IF EXISTS dev_ecommerce CASCADE")



In [0]:
def delete_folders(path):
    try:
        dbutils.fs.rm(path, recurse=True)
        print(f"Folder {path} eliminado.")
    except Exception:
        print(f"No existe folder: {path}")
        pass
    
def create_schema(schema):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS dev_ecommerce.{schema}")
    

In [0]:
# Eliminar todas las carpetas de los containers

#dbutils.fs.rm("abfss://bronze@adlsproyecto2025.dfs.core.windows.net/sales_orders/", recurse=True)

for schema in schemas_list:
    delete_folders(f"abfss://{schema}@adlsproyecto2025.dfs.core.windows.net/sales_orders/")




In [0]:
# crea el catálogo
spark.sql("CREATE CATALOG IF NOT EXISTS dev_ecommerce")

# crea el esquema bronze 
for schema in schemas_list:
    create_schema(schema)



In [0]:
print("Se ha creado el catálogo y los esquemas")